In [ ]:
import pandas as pd
import numpy as np

In [ ]:
FH=pd.read_csv('./data/CleanedPlayerData/MergedHitters.csv')
FP=pd.read_csv('./data/CleanedPlayerData/MergedPitchers.csv')

In [ ]:
pctcols=[i for i in FH.columns if '%' in i]
for i in pctcols:
    FH[i]=FH[i].str.replace('%','').astype(float)/100
pctcols=[i for i in FP.columns if '%' in i]
for i in pctcols:
    FP[i]=FP[i].str.replace('%','').astype(float)/100
FH['hHR/FB']=FH['hHR/FB'].str.replace('%','').astype(float)/100
FP['pHR/FB']=FP['pHR/FB'].str.replace('%','').astype(float)/100

In [ ]:
FH.columns

In [ ]:
FP.columns

In [ ]:
def construct_input(atts,parr,harr):
    atts[2]=float(atts[2])
    features=['stand','p_throws', 'season',
          'hBB%','hK%','hSpd','hPull%','hGB%','hLD%','hFB%',
          'havg_hit_angle','hmax_hit_speed', 'havg_hit_speed', 'hfbld','hgb','hev95percent',
          'pBB%','pK%','pPull%','pGB%','pLD%','pFB%',
          'pavg_hit_angle','pmax_hit_speed', 'pavg_hit_speed', 'pfbld','pgb','pev95percent']
    hfeats=['hBB%','hK%','hSpd','hPull%','hGB%','hLD%','hFB%',
          'havg_hit_angle','hmax_hit_speed', 'havg_hit_speed', 'hfbld','hgb','hev95percent']
    pfeats=['pBB%','pK%','pPull%','pGB%','pLD%','pFB%',
          'pavg_hit_angle','pmax_hit_speed', 'pavg_hit_speed', 'pfbld','pgb','pev95percent']
    data_z=np.zeros((1,3))
    df=pd.DataFrame(data=data_z,columns=features[0:3])
    df.loc[0,'stand']=atts[0]
    df.loc[0,'p_throws']=atts[1]
    df.loc[0,'season']=atts[2]
    for i in hfeats:
        df[i]=harr[i].values
    for i in pfeats:
        df[i]=parr[i].values
    return df

In [ ]:
def simAB(pitcher,hitter,ispRH=True,isbRH=True,pseason=2019,hseason=2019,season=2019,output='probs'):
    pitcher=FP[(FP['pName']==pitcher)&(FP['pseason']==pseason)]
    hitter=FH[(FH['hName']==hitter)&(FH['hseason']==hseason)]
    atts=[float(int(isbRH)),float(int(ispRH)),season]
    feats_arr=construct_input(atts,pitcher,hitter)
    feats_sc=ss.transform(feats_arr)
    preds=model.predict(feats_sc)
    preds_clean=np.round(preds,3)
    if output=='probs':
        cols=['GB', '2B', '1B', 'K', 'BB', 'PU', 'FB', 'LD', 'HR', 'HBP', '3B']
        df=pd.DataFrame(data=preds_clean,columns=cols)
    return df

In [ ]:
simAB('Corey Kluber','Miguel Sano')